In [14]:
import pandas as pd
import numpy as np
import jieba
from gensim.models import Word2Vec
from gensim import models

In [15]:
movie_comments = pd.read_csv('./movie_comments.csv')

In [16]:
movie_comments.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [17]:
len(movie_comments)

261497

In [18]:
movie_comments.dropna(subset=['comment','star'],inplace=True)

In [19]:
len(movie_comments)

261495

In [20]:
set(movie_comments['star'])

{1, '1', '2', 2, 3, '3', 4, '4', 5, '5', 'star'}

In [21]:
movie_comments[movie_comments['star']=='star']

,id,link,name,comment,star
568,id,link,name,comment,star


In [22]:
movie_comments.drop([568],inplace=True)

In [23]:
len(movie_comments.star)

261494

In [24]:
set(movie_comments['star'])

{1, '1', '2', 2, 3, '3', 4, '4', 5, '5'}

In [25]:
movie_comments['star']

0         1
1         2
2         2
3         4
4         1
         ..
261492    3
261493    3
261494    2
261495    3
261496    3
Name: star, Length: 261494, dtype: object

In [26]:
movie_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261494 entries, 0 to 261496
Data columns (total 5 columns):
id         261494 non-null object
link       261494 non-null object
name       261494 non-null object
comment    261494 non-null object
star       261494 non-null object
dtypes: object(5)
memory usage: 12.0+ MB


In [27]:
y = [int(star) for star in movie_comments.star]

In [28]:
comments = movie_comments.comment

In [29]:
def cut(sen):
    return ' '.join(jieba.cut(sen))

In [30]:
import re
def token(str):
    return re.findall(r'[\d|\w]+',str)

In [31]:
comments_cut = [cut(''.join(token(comment))) for comment in comments]

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/hw/z_fcnbnd33x0l3cqd8lp6_t00000gn/T/jieba.cache
Loading model cost 1.051 seconds.
Prefix dict has been built succesfully.


In [32]:
comments_cut[1].split()

['首映礼',
 '看',
 '的',
 '太',
 '恐怖',
 '了',
 '这个',
 '电影',
 '不讲道理',
 '的',
 '完全',
 '就是',
 '吴京',
 '在',
 '实现',
 '他',
 '这个',
 '小',
 '粉红',
 '的',
 '英雄',
 '梦',
 '各种',
 '装备',
 '轮番',
 '上场',
 '视',
 '物理',
 '逻辑',
 '于',
 '不顾',
 '不得不',
 '说',
 '有钱',
 '真',
 '好',
 '随意',
 '胡闹']

In [33]:
len(y)

261494

In [34]:
len(comments)

261494

In [35]:
len(set(movie_comments.name.values))

2759

In [36]:
def process(content):
    star = int(content['star'])
    com = token(content['comment'])
    
    return pd.Series([com,star],index=['comments','y'])

In [37]:
movie_comments_2= movie_comments.apply(process,axis=1)

In [38]:
movie_comments_2.to_csv('./douban.csv', index=False)

In [300]:
movie_comments_2.head()

,comments,y
0,"[吴京意淫到了脑残的地步, 看了恶心想吐]",1
1,"[首映礼看的, 太恐怖了这个电影, 不讲道理的, 完全就是吴京在实现他这个小粉红的英雄梦, ...",2
2,"[吴京的炒作水平不输冯小刚, 但小刚至少不会用主旋律来炒作, 吴京让人看了不舒服, 为了主旋...",2
3,"[凭良心说, 好看到不像, 战狼1, 的续集, 完虐, 湄公河行动]",4
4,[中二得很],1


In [43]:
set(movie_comments_2[movie_comments['name']=='战狼2'].y.values)

{1, 2, 3, 4, 5}

In [44]:
set(movie_comments_2['y'])

{1, 2, 3, 4, 5}

In [45]:
classes = []
for name in set(movie_comments['name'].values):
    class_ = set(movie_comments_2[movie_comments['name']==name].y.values)
    classes.append(len(class_))
len(classes)

2759

In [46]:
classes.count(5)

1387

# 训练句向量

In [47]:
model = models.Word2Vec.load('/Users/zhangjie/Documents/machinelearning/att/nplAssignments-local/assignment-4-/word2vec.model')

In [48]:
model.wv['吴京'].shape

(200,)

In [49]:
len(model.wv.vocab)

686470

In [50]:
model.wv.most_similar('好看')

[('好笑', 0.793709397315979),
 ('难看', 0.7650017142295837),
 ('吓人', 0.7606807351112366),
 ('挑剔', 0.7506513595581055),
 ('厉害', 0.7499526739120483),
 ('好听', 0.7471263408660889),
 ('漂亮', 0.7462058067321777),
 ('有意思', 0.7397966384887695),
 ('难听', 0.7330186367034912),
 ('好吃', 0.7326034307479858)]

In [51]:
def sentence_vec(sentences):
    sentence_vecs = []
    for sentence in sentences:
        s=0
        sentence_vec = np.zeros(200)
        words = sentence.split()
        for word in words:
            if word in model.wv.vocab:
                sentence_vec+=model.wv[word]
                s+=1
            else:
                continue
        if (s !=0):
            sentence_vec=sentence_vec/s
        else:
            sentence_vec=np.zeros(200)
        sentence_vecs.append(sentence_vec)
    return sentence_vecs

In [52]:
sentence_vecs=sentence_vec(comments_cut)

In [57]:
sentence_vecs[0].shape

(200,)

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train,X_test, y_train, y_test =train_test_split(sentence_vecs,y,test_size=0.2, random_state=0)

In [60]:
len(X_train),len(y_train),len(X_test),len(y_test)

(209195, 209195, 52299, 52299)

In [61]:
import numpy as np
X_train_arr = np.array(X_train, dtype='float32')
y_train_arr = np.array(y_train, dtype='float32')
X_test_arr = np.array(X_test, dtype='float32')
y_test_arr = np.array(y_test, dtype='float32')

In [62]:
set(y_train_arr)

{1.0, 2.0, 3.0, 4.0, 5.0}

In [63]:
X_train_arr.shape

(209195, 200)

# LogisticRegression

In [64]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train,y_train)

/anaconda3/envs/NPL-3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/NPL-3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [65]:
clf.score(X_train,y_train)

0.37261406821386744

In [66]:
clf.score(X_test,y_test)

0.37268399013365455

In [67]:
clf_2 = LogisticRegression().fit(X_train_arr,y_train_arr)

/anaconda3/envs/NPL-3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/NPL-3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [68]:
clf_2.score(X_train_arr,y_train_arr)

0.3726523100456512

In [69]:
clf_2.score(X_test_arr,y_test_arr)

0.37268399013365455

# tensorFlow

In [70]:
numble_labels=6
def reformat(labels):
    labels = (np.arange(numble_labels)==labels[:,None]).astype(np.float32)
    return labels

In [77]:
y_train_arr_re = reformat(y_train_arr)
y_test_arr_re = reformat(y_test_arr)

In [78]:
y_train_arr[0],y_train_arr_re[0],y_train_arr_re.shape,y_test_arr_re.shape

(4.0, array([0., 0., 0., 0., 1., 0.], dtype=float32), (209195, 6), (52299, 6))

In [73]:
import tensorflow as tf

num_nodes= 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 200))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, numble_labels))
    tf_test_dataset = tf.constant(X_test_arr)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([200, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, numble_labels]))
    biases_2 = tf.Variable(tf.zeros([numble_labels]))

    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1) # add relu layer
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_2, labels = tf_train_labels))
    
    # Loss function using L2 Regularization
    regularizer = tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_1)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [79]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [80]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (y_train_arr_re.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = X_train_arr[offset:(offset + batch_size), :]
        batch_labels = y_train_arr_re[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Batch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), y_test_arr_re)))

Initialized
Minibatch loss at step 0: 1086.5985107421875
Batch accuracy: 9.4
Minibatch loss at step 500: 9.317553520202637
Batch accuracy: 34.4
Minibatch loss at step 1000: 1.4707963466644287
Batch accuracy: 38.3
Minibatch loss at step 1500: 1.5642917156219482
Batch accuracy: 23.4
Minibatch loss at step 2000: 1.405497670173645
Batch accuracy: 34.4
Minibatch loss at step 2500: 1.4934420585632324
Batch accuracy: 26.6
Minibatch loss at step 3000: 1.4294017553329468
Batch accuracy: 28.1
Test accuracy: 35.0


# fastText

In [81]:
import fasttext
import os

In [82]:
movie_douban = pd.read_csv('./douban.csv',lineterminator='\n')

In [83]:
movie_douban.head()

,comments,y
0,"['吴京意淫到了脑残的地步', '看了恶心想吐']",1
1,"['首映礼看的', '太恐怖了这个电影', '不讲道理的', '完全就是吴京在实现他这个小粉...",2
2,"['吴京的炒作水平不输冯小刚', '但小刚至少不会用主旋律来炒作', '吴京让人看了不舒服'...",2
3,"['凭良心说', '好看到不像', '战狼1', '的续集', '完虐', '湄公河行动']",4
4,['中二得很'],1


In [84]:
def proprecessing(df, dataset):
    with open(os.path.join('./', dataset), 'w', encoding='utf8') as f:
        for comment, label in df.values:
            line = '__label__'+str(int(label))+' '+comment+'\n'
            f.write(line)

In [85]:
train ,test= train_test_split(movie_douban,random_state=2019)

In [93]:
train.head()

,comments,y
65617,"['幸好是试映会看的', '预告片欺骗性很强', '正片完成度很低']",2
116012,"['我没有信仰所以相信第二个故事', '动物残杀情节很短', '大篇幅讲老虎和派的故事', ...",5
194876,"['小机器人真是可爱哈', '话说那孩子就是飞行天才']",4
56127,['出乎意料的好看'],5
107822,"['开篇很有意思', '越到后面越俗套', 'TONY哥的手指是绝对亮点']",3


In [94]:
train.values

array([["['幸好是试映会看的', '预告片欺骗性很强', '正片完成度很低']", 2],
       ["['我没有信仰所以相信第二个故事', '动物残杀情节很短', '大篇幅讲老虎和派的故事', '其实残酷的现实很快发生', '剩下长达很久的内心斗争']",
        5],
       ["['小机器人真是可爱哈', '话说那孩子就是飞行天才']", 4],
       ...,
       ["['冲着演员和题材去的', '没的说', '极力推荐之']", 5],
       ["['很好看', '但我没哭', '就被同事说太冷血']", 4],
       ["['好可怕好惊悚啊']", 3]], dtype=object)

In [95]:
test.values

array([["['Jake好好一个帅哥就这样在通往实力派的道路上成了freak', '电影有多有趣的地方', '黑暗到觉得那些个本来可以成为幽默元素的讽刺调侃', '浮夸的主角台词', '都浓重成了墨']",
        4],
       ["['并未觉得开篇惊艳', '实在有些刻意', '从死神来了', '灵数13', '这类影片', '没少写这种连环意外和精神紧张的事情', '包括后来几次意外也有些牵强', '禁不起推敲', '人物也都半吊子', '这类偶然事件的串联', '比较喜欢的还是', '蝴蝶振翅']",
        1],
       ["['还行']", 4],
       ...,
       ["['人生多么复杂多么艰难', '每个人都有难以言说的地方', 'Ted在被告席上朝证人台上的Joanna摇头做出No的口型的时候', '我突然觉得这个男人实在太好了', 'It', 'has', 'to', 'do', 'with', 'constancy', 'it', 'has', 'to', 'do', 'with', 'patience', 'it', 'has', 'to', 'do', 'with', 'listening', 'and', 'pretending', 'to', 'listen', 'when', 'you', 'can', 't', 'even', 'listen', 'any', 'more']",
        5],
       ["['网上有篇江一燕的软文', '这两个应该对比来看', '才觉得人生张力多么巨大', '有多么荒诞反讽']", 1],
       ["['颤栗']", 4]], dtype=object)

In [96]:
train.shape,test.shape

((196120, 2), (65374, 2))

In [97]:
proprecessing(train, 'douban.train')

In [98]:
proprecessing(test, 'douban.test')

In [91]:
model = fasttext.train_supervised('./douban.train', verbose=5, lr=1e-2, epoch=20)

In [99]:
model.test('./douban.test')

(65374, 0.4563740936763851, 0.4563740936763851)

In [100]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.8f}".format(1, p))
    print("R@{}\t{:.8f}".format(1, r))

In [101]:
print_results(*model.test('./douban.test'))

N	65374
P@1	0.45637409
R@1	0.45637409


In [102]:
model.predict("还行")[0][0][-1]

'3'

In [103]:
pred_test = []
for x, _ in test.values:
    pred = model.predict(x)[0][0][-1]
    pred_test.append(int(pred))
pred_test = np.array(pred_test)

In [104]:
pred_test

array([3, 1, 3, ..., 5, 3, 4])